In [3]:
import os
import pandas as pd
from datetime import datetime

# import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


def extract_scalars_from_event_file(event_file):
    event_acc = EventAccumulator(event_file)
    event_acc.Reload()
    

    tags = event_acc.Tags()["scalars"]

    all_scalars = {}
    for tag in tags:
        events = event_acc.Scalars(tag)
        values = [event.value for event in events]
        steps = [event.step for event in events]

        timestamps = [pd.to_datetime(event.wall_time, unit='s') for event in events]
        all_scalars[tag] = pd.DataFrame({"step": steps, "value": values, "timestamp": timestamps})

    return all_scalars


files_t = []
def event_files_to_dataframe(directory):
    all_scalars_dfs = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.startswith("events.out.tfevents"):
                event_file = os.path.join(root, file)

                file_from = pd.to_datetime(file.split(".")[3], unit='s')
                files_t.append(file_from)

                # print(file_from)
                if file_from < date_obj:
                    continue

                scalars = extract_scalars_from_event_file(event_file)
                for tag, df in scalars.items():
                    df["tag"] = tag
                    all_scalars_dfs.append(df)

    all_scalars_df = pd.concat(all_scalars_dfs, ignore_index=True)
    return all_scalars_df

df_all = []
for i in range(10):
    # Directory containing your event files
    log_dir = f"./dfl_10_fully_connected/participant_{i}"
    date_str = "02/07/2024 9:10:00" # minus two hours for timezone ( given in fedstellar)

    date_obj = datetime.strptime(date_str, "%d/%m/%Y %H:%M:%S")

    df = event_files_to_dataframe(log_dir)
    print(df.head())

    df.sort_values(by="timestamp", inplace=True)

    files_t.sort(reverse=True)
    print(files_t[0], files_t[-1])

    df[df["tag"]=='Test/Loss']["timestamp"].max()

    test_df = df[df["tag"].str.startswith("Test/")].groupby("tag").last().copy()
    print(test_df)
    test_df["participant"] = f"participant_{i}"
    df_all.append(test_df)

   step     value                     timestamp         tag
0    19  1.561008 2024-07-02 09:13:01.258073856  Train/Loss
1    39  1.173001 2024-07-02 09:13:02.378592256  Train/Loss
2    59  1.082819 2024-07-02 09:13:05.697747712  Train/Loss
3    79  0.563365 2024-07-02 09:13:06.801960704  Train/Loss
4    99  0.412374 2024-07-02 09:13:08.089794048  Train/Loss
2024-07-02 09:16:32 2024-07-02 09:11:21
                step     value                     timestamp
tag                                                         
Test/Accuracy   1837  0.936925 2024-07-02 09:16:31.281275136
Test/F1Score    1837  0.934486 2024-07-02 09:16:31.281404416
Test/Loss       1837  0.079109 2024-07-02 09:16:31.281161216
Test/Precision  1837  0.938760 2024-07-02 09:16:31.281322752
Test/Recall     1837  0.936925 2024-07-02 09:16:31.281363456
   step      value                     timestamp                    tag
0     0   0.000000 2024-07-02 09:10:28.866646272                  Round
1    14  51.700001 2024-07-02

In [4]:
concat_df = pd.concat(df_all)
concat_df

,step,value,timestamp,participant
tag,,,,
Test/Accuracy,1837,0.936925,2024-07-02 09:16:31.281275136,participant_0
Test/F1Score,1837,0.934486,2024-07-02 09:16:31.281404416,participant_0
Test/Loss,1837,0.079109,2024-07-02 09:16:31.281161216,participant_0
Test/Precision,1837,0.938760,2024-07-02 09:16:31.281322752,participant_0
Test/Recall,1837,0.936925,2024-07-02 09:16:31.281363456,participant_0
Test/Accuracy,1640,0.965468,2024-07-02 09:16:31.437102848,participant_1
Test/F1Score,1640,0.961161,2024-07-02 09:16:31.437172736,participant_1
Test/Loss,1640,0.073815,2024-07-02 09:16:31.437011456,participant_1
Test/Precision,1640,0.964322,2024-07-02 09:16:31.437129984,participant_1


In [5]:
concat_df.groupby("tag").mean()

/tmp/ipykernel_38683/1972488931.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").mean()


,step,value
tag,,
Test/Accuracy,1845.1,0.950626
Test/F1Score,1845.1,0.946641
Test/Loss,1845.1,0.070412
Test/Precision,1845.1,0.949778
Test/Recall,1845.1,0.950626


In [6]:
concat_df.to_csv("results_csvs/dfl_10_fully_connected.csv")